In [1]:
%reload_ext autoreload
%autoreload 2

In [5]:
import os, sys
import re
import json
import glob
import datetime
from collections import Counter

import pandas as pd
from matplotlib import pyplot as plt
# import seaborn as sns

# from nltk.corpus import stopwords
# from wordcloud import WordCloud
# Adding the 'src' folder to the Python path
sys.path.append('../src')  
# Import json file loader method from loader.py
from loader import SlackDataLoader  

slackDataLoader = SlackDataLoader('../data/anonymized')

### Columns we can get from a slack message<br>

message_type, message_content, sender_id, time_sent, message_distribution, time_thread_start, reply_count, reply_user_count, time_thread_end, reply_users

In [31]:
directory_path = '../data/anonymized/'
# Get all channels list of directories 
directories = [name for name in os.listdir(directory_path) if os.path.isdir(os.path.join(directory_path, name))]
directories

['ab_test-group',
 'adludios-challange',
 'all-broadcast',
 'all-career-exercises',
 'all-community-building',
 'all-de-week12',
 'all-ideas',
 'all-ml-week12',
 'all-resources',
 'all-technical-support',
 'all-web3-week12',
 'all-week1',
 'all-week10',
 'all-week11',
 'all-week12',
 'all-week2',
 'all-week3',
 'all-week4',
 'all-week5',
 'all-week6',
 'all-week7',
 'all-week8',
 'all-week9',
 'batch6_week4_studygroup',
 'chang-w11',
 'data-engineering',
 'dsa-sql',
 'gokada-challenge-presentation',
 'happy-new-year-study-group',
 'kafka_de',
 'machine-learning',
 'random',
 'study-group',
 'team-10',
 'tenx-bot',
 'week-11-group4',
 'week-2-group-8',
 'week2-group',
 'week4-teamwork']

In [33]:
# get the first channel slack data
slackDataLoader.slack_parser('../data/anonymized/ab_test-group/')

,msg_type,msg_content,sender_name,msg_sent_time,msg_dist_type,time_thread_start,reply_count,reply_users_count,reply_users,tm_thread_end,channel
0,message,"Hi guys,\nI thought having a group chat would ...",Anita Rodriguez,1661766066.863109,text,0,0,0,0,0,
1,message,good!!,Michael Gonzalez,1661766159.852649,text,0,0,0,0,0,
2,message,Thanks <@U03U9FWPNCE>,Robert Carter,1661766226.215889,text,0,0,0,0,0,
3,message,Nice <@U03U9FWPNCE>,Phillip Atkins,1661766241.597379,text,0,0,0,0,0,
4,message,I have been missing deadlines and I am trying ...,Anita Rodriguez,1661766295.939119,text,0,0,0,0,0,
...,...,...,...,...,...,...,...,...,...,...,...
541,message,What did they say?,Phillip Atkins,1662368104.629779,text,0,0,0,0,0,
542,message,Nothing!\nthey didn't notice!,Anita Rodriguez,1662368288.166039,text,0,0,0,0,0,
543,message,Nah thats even worse. Are we the only group th...,Phillip Atkins,1662368563.529889,text,0,0,0,0,0,
544,message,Not sure!,Anita Rodriguez,1662368582.472009,text,0,0,0,0,0,


In [ ]:
max_reply_count = 0
for channel in directories:
    data = slackDataLoader.slack_parser(f"../data/anonymized/{channel}/")
    df = pd.DataFrame(data)
    # df.loc[df['reply_count'].idxmax()]
    if  df['reply_count'].max() > max:
        max_reply_count = df['reply_count'].max()
    
max_reply_count

In [ ]:
data = slackDataLoader.slack_parser('../data/anonymized/')
df = pd.DataFrame(data)
df.loc[df['reply_count'].idxmin()]

In [39]:
# List to store the results for each metric
top_bottom_results = []

for channel in directories:
    data = slackDataLoader.slack_parser(f"../data/anonymized/{channel}/")
    df = pd.DataFrame(data)
    
    # Top 10 messages with most reply count
    top_reply_count_users = df.groupby('msg_content')['reply_count'].sum().nlargest(10)
    
    # Store the results in a dictionary
    result = {
        'Channel': channel,
        'Top Message by reply': top_reply_count_users,
    }
    
    # Append the dictionary to the list
    top_bottom_results.append(result)


# top_bottom_results
# Print the results for each channel
for result in top_bottom_results:
    print("Channel:", result['Channel'])
    print("Top Message by reply:", result['Top Message by reply'])
    print("--------------------")

Channel: ab_test-group
Top Message by reply: msg_content
Are we supposed to do decision tree too?                                                                                                                                                                                                                                          4
for script? <@U03UG4Q7V42>                                                                                                                                                                                                                                                        2
What does it mean when I get this message:\nAuthentication successful.\nA\ndata\AdSmartABdata.csv1 file added and 1 file fetched\n???                                                                                                                                             1
do we have CML on this task?                                                                                       

From a single slack message, we can get <br>

1. The message<br>
2. Type (message, file, link, etc)<br>
3. The sender_id (assigned by slack)<br>
4. The time the message was sent<br>
5. The team (i don't know what that is now)<br>
6. The type of the message (broadcast message, inhouse, just messgae)<br>
7. The thread the message generated (from here we can go):<br>
    7.1 Text/content of the message<br>
    7.2 The thread time of the message<br>
    7.3 The thread count (reply count)<br>
    7.4 The number of user that reply the message (count of users that participated in the thread)<br>
    7.5 The time the last thread message was sent <br>
    7.6 The users that participated in the thread (their ids are stored as well)<br>

In [ ]:
def map_userid_2_realname(user_profile: dict, comm_dict: dict, plot=False):
    """
    map slack_id to realnames
    user_profile: a dictionary that contains users info such as real_names
    comm_dict: a dictionary that contains slack_id and total_message sent by that slack_id
    """
    user_dict = {} # to store the id
    real_name = [] # to store the real name
    ac_comm_dict = {} # to store the mapping
    count = 0
    # collect all the real names
    for i in range(len(user_profile['profile'])):
        real_name.append(dict(user_profile['profile'])[i]['real_name'])

    # loop the slack ids
    for i in user_profile['id']:
        user_dict[i] = real_name[count]
        count += 1

    # to store mapping
    for i in comm_dict:
        if i in user_dict:
            ac_comm_dict[user_dict[i]] = comm_dict[i]

    ac_comm_dict = pd.DataFrame(data= zip(ac_comm_dict.keys(), ac_comm_dict.values()),
    columns=['LearnerName', '# of Msg sent in Threads']).sort_values(by='# of Msg sent in Threads', ascending=False)
    
    if plot:
        ac_comm_dict.plot.bar(figsize=(15, 7.5), x='LearnerName', y='# of Msg sent in Threads')
        plt.title('Student based on Message sent in thread', size=20)
        
    return ac_comm_dict

In [ ]:
def draw_avg_reply_count(data, channel='Random'):
    """who commands many reply?"""

    data.groupby('sender_name')['reply_count'].mean().sort_values(ascending=False)[:20]\
        .plot(kind='bar', figsize=(15,7.5));
    plt.title(f'Average Number of reply count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_avg_reply_users_count(data, channel='Random'):
    """who commands many user reply?"""

    data.groupby('sender_name')['reply_users_count'].mean().sort_values(ascending=False)[:20].plot(kind='bar',
     figsize=(15,7.5));
    plt.title(f'Average Number of reply user count per Sender in #{channel}', size=20, fontweight='bold')
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

def draw_wordcloud(msg_content, week):    
    # word cloud visualization
    allWords = ' '.join([twts for twts in msg_content])
    wordCloud = WordCloud(background_color='#975429', width=500, height=300, random_state=21, max_words=500, mode='RGBA',
                            max_font_size=140, stopwords=stopwords.words('english')).generate(allWords)
    plt.figure(figsize=(15, 7.5))
    plt.imshow(wordCloud, interpolation="bilinear")
    plt.axis('off')
    plt.tight_layout()
    plt.title(f'WordCloud for {week}', size=30)
    plt.show()

def draw_user_reaction(data, channel='General'):
    data.groupby('sender_name')[['reply_count', 'reply_users_count']].sum()\
        .sort_values(by='reply_count',ascending=False)[:10].plot(kind='bar', figsize=(15, 7.5))
    plt.title(f'User with the most reaction in #{channel}', size=25);
    plt.xlabel("Sender Name", size=18); plt.ylabel("Frequency", size=18);
    plt.xticks(size=14); plt.yticks(size=14);
    plt.show()

## Insight Extraction

Below are some useful questions to answer. Feel free to explore to answer other interesting questions that may be of help to get insight about student's behaviour, need, and future performance 

In [3]:
# which user has the highest number of reply counts?


In [ ]:
# Visualize reply counts per user per channel

In [ ]:
# what is the time range of the day that most messages are sent?


In [ ]:
# what kind of messages are replied faster than others?

In [ ]:
# Relationship between # of messages and # of reactions

In [ ]:
# Classify messages into different categories such as questions, answers, comments, etc.

In [ ]:
# Which users got the most reactions?

In [ ]:
# Model topics mentioned in the channel

In [ ]:
# What are the topics that got the most reactions?

### Harder questions to look into

In [ ]:
# Based on messages, reactions, references shared, and other relevant data such as classification of questions into techical question, comment, answer, aorder stu the python, statistics, and sql skill level of a user?